# 3. 모델 생성

## 3.1 데이터 읽기 및 확인

In [1]:
# 데이터 로드
import pandas as pd

bank_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/전처리/chap3-4/data/bank-prep.csv')
bank_df.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,y,divorced,married,single,primary,secondary,tertiary,cellular,telephone,unknown,apr,aug,dec,feb,jan,jul,jun,mar,may,nov,oct,sep
0,58,0,2143,1,0,5,261,1,-1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,36,0,265,1,1,5,348,1,-1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,25,0,-7,1,0,5,365,1,-1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,53,0,-3,0,0,5,1666,1,-1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,24,0,-103,1,1,5,145,1,-1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


## 3.2 불균형 데이터의 균형화

- 목적변수 ' y '의 경우, yes가 no에 비해 압도적으로 데이터 건수가 적음
- 다수 클래스의 데이터를 섞고 소수 클래스와 동일한 건수의 데이터를 선택 → [언더샘플링]
- 소수 클래스의 데이터를 다수 클래스의 데이터 건수와 맞춤 → [오버샘플링]
- imbalanced-learn 패키지: 언더샘플링 혹은 오버샘플링을 위한 기능 제공

In [2]:
# imbalanced-learn 패키지 설치 유무 확인
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.4.3
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: None
Author-email: None
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: scipy, scikit-learn, numpy
Required-by: imblearn


In [3]:
bank_df_new = bank_df.copy()

In [5]:
# 불균형 데이터셋에 대한 언더 샘플링
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

X = np.array(bank_df_new.drop('y', axis = 1)) # 'y' 외의 모든 항목을 설명변수
Y = np.array(bank_df_new[['y']]) # 'y' 항목을 목적변수
print(np.sum(Y == 1), np.sum(Y == 0))

sampler = RandomUnderSampler(random_state = 42)
X, Y = sampler.fit_resample(X, Y)
print(np.sum(Y == 1), np.sum(Y == 0))

820 6113
820 820


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


- ' y ' 의 값이 ' 1(yes) '인 갯수 820에 맞춰 언더샘플링 되었음을 확인 가능

## 3.3 결정 트리 모델의 작성 및 검증

In [10]:
# 결정 트리 알고리즘을 사용한 모델 검증
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

# K-Fold 교차검증
kf = KFold(n_splits = 10, shuffle = True)
scores = []

# 데이터 셔플한 후, 훈련 데이터와 테스트 데이터로 분할
for train_id, test_id in kf.split(X):
  x = X[train_id]
  y = Y[train_id]
  clf = tree.DecisionTreeClassifier()
  clf.fit(x, y)

  pred_y = clf.predict(X[test_id])
  score = accuracy_score(Y[test_id], pred_y)
  scores.append(score)

scores = np.array(scores)
print(scores.mean(), scores.std())

0.7682926829268293 0.02685697017749422


In [12]:
# 재현율, 적합도 확인
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(recall_score(Y[test_id], pred_y)) # 재현율
print(precision_score(Y[test_id], pred_y)) # 적합도

0.7368421052631579
0.7466666666666667


- 분석 목표가 ' 계약 확률이 높은 고객을 찾아내는 것 ' 이므로, 재현율을 좀 더 높일 필요가 있음

## 3.4 파라미터 최적화

In [13]:
# 파라미터 기본값 확인
print(clf)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


- 지금의 파라미터가 최적이라고 장담하기 힘듦 → 그리드 서치 사용, 최적의 파라미터 조합 탐색

In [14]:
# 그리드 서치 사용, 모델 성능 개선
from sklearn.model_selection import GridSearchCV

params = {
    'criterion': ['entropy'],
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50]
}

clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params,
                      cv = KFold(n_splits = 10, shuffle = True), scoring = 'accuracy')

clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             pa

In [15]:
# 가장 높은 성능일 때의 모델에 대한 파라미터 조합 확인
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.7939024390243903
{'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 20}


In [16]:
# 모델 완성 및 영향력 높은 변수 확인
clf_best = tree.DecisionTreeClassifier(
    criterion = 'entropy', max_depth = 10, min_samples_leaf = 20
)
clf_best.fit(X, Y)

print(clf_best.feature_importances_) # 모델 작성에 사용한 모든 결과에 대한 영향력 표시

[0.01501975 0.         0.04462147 0.09361932 0.01618605 0.04211927
 0.54413625 0.00306831 0.06565549 0.03995862 0.         0.00560226
 0.         0.         0.00305009 0.         0.         0.
 0.07052362 0.         0.013052   0.         0.         0.
 0.01208667 0.         0.         0.03130083 0.         0.
 0.        ]


- 좀 더 깊이있는 전처리 → 기존변수로부터 신규변수 작성 → 좀 더 좋은 모델이 될 수 있음